In [106]:
#sql to pandas
import pandas
from student.models import Grade, Attendance
from django.db import connection
import gviz_api
import math


#for notebook only
from bokeh.charts import show
from bokeh.io import output_notebook



In [107]:
student_id=4


In [113]:
def getPoints(x):
    # if no grade for that subject at that date
    if math.isnan(x):
        # just return it untouched
        return x
    # but, if not, return the points
    elif x:
        if x>=90:
            return 4
        elif x>=80:
            return 3
        elif x>=70:
            return 2
        elif x>=60:
            return 1
        else:
            return 0
    # and leave everything else
    else:
        return 
    
all_grades_sql = "SELECT grade, grade_date, subject_name  FROM student_grade, student_subject  \
           WHERE student_grade.student_id = '%s' \
            AND student_grade.subject_id=student_subject.subject_id\
            ORDER BY date(grade_date) DESC" %(student_id )

df_all_grades = pandas.read_sql(all_grades_sql, con=connection)
df_grades_indexed=df_all_grades.pivot(index='grade_date', columns='subject_name', values='grade')
df_points=df_grades_indexed.applymap(getPoints)
df_points['gpa']=df_points.mean(axis=1)
df_points=df_points.reset_index()
gpa_values=df_points[['grade_date', 'gpa']].values
gpa_desc=[("grade_date", "date", "Date" ),
          ("gpa", "number", "GPA")]
gpa_data_table=gviz_api.DataTable(gpa_desc)
gpa_data_table.LoadData(gpa_values)
gpa_json=gpa_data_table.ToJSon()
#will be the GPA of the most recently entered grades
#maybe change this to be the 4 most recent grades per subject!
current_gpa=df_points.sort_values('grade_date',0,False)['gpa'].iloc[0]
current_gpa


4.0

In [115]:
attend_pct=round(Attendance.objects.filter(student_id="%s"%(student_id)).order_by('-attend_date')[1].calc_pct())
attend_pct

97.0